In [1]:
import psycopg2
from imessage_reader import fetch_data
from os.path import expanduser
import sys
import sqlite3
import datetime as dt
import plotly.graph_objects as go
import pandas as pd


In [2]:
def postgre_exe(query):
    cursor.execute(query)
    cursor.execute("COMMIT")

In [3]:
db = psycopg2.connect(host = 'localhost', dbname = 'ledger', user = 'mato', port = 5432)
cursor = db.cursor()

In [22]:
db.commit()

In [23]:
# query = """SELECT SUM(AMOUNTS) / EXTRACT(DAY FROM NOW()) AS AVERAGE_PAY, EXTRACT(DAY FROM NOW()) FROM LEDGER.TRANSACTION WHERE TRANS_DATE >= DATE_TRUNC('MONTH', CURRENT_DATE) AND TYPE = 2;"""
# query = """
#         SELECT
#             TRANS_DATE,
#             TO_CHAR(TRANS_DATE, 'D') AS TRANS_DATE_NUM
#             FROM LEDGER.TRANSACTION;
#         """
# query = """
#         WITH PAYS_WEEK AS
#             ( SELECT TRANS_ID, 
#                 TRANS_DATE, 
#                 CASE WHEN TO_CHAR(TRANS_DATE, 'D') IN ('1', '7') THEN 'WEEKEND' ELSE 'WEEKDAY' END AS WEEK, 
#                 AMOUNTS 
#               FROM LEDGER.TRANSACTION
#               WHERE TRANS_DATE >= DATE_TRUNC('MONTH', CURRENT_DATE) AND TYPE = 2)
#         SELECT 
#             WEEK,
#             SUM(AMOUNTS),
#             COUNT(DISTINCT(WEEK))
#         FROM PAYS_WEEK
#         GROUP BY WEEK;
#         """
query = """
        WITH PAYS_PER_DAY AS
            (
                SELECT 
                    TRANS_DATE,
                    SUM(AMOUNTS)        AS AMOUNTS_PER_DAY
                FROM
                    LEDGER.TRANSACTION
                WHERE
                    TRANS_DATE >= DATE_TRUNC('MONTH', CURRENT_DATE) AND TYPE = 2
                GROUP BY TRANS_DATE
            )
        SELECT
            CASE WHEN TO_CHAR(TRANS_DATE, 'D') IN ('1', '7') THEN 'WEEKEND' ELSE 'WEEKDAY' END AS WEEK, 
            AVG(AMOUNTS_PER_DAY)
        FROM PAYS_PER_DAY
        GROUP BY WEEK;
        """
cursor.execute(query)
results = cursor.fetchall()
results

[('WEEKDAY', Decimal('35455.176470588235')),
 ('WEEKEND', Decimal('128341.666666666667'))]

In [24]:
int(results[0][0])

60256

In [14]:
query = """SELECT * FROM LEDGER.TRANSACTION;"""
cursor.execute(query)
results = cursor.fetchall()

In [16]:
results[0]

(1,
 datetime.date(2023, 1, 1),
 datetime.time(10, 11),
 'Sun',
 2,
 1,
 4900,
 '네이버플러스구독요금',
 '네이버파이낸셜',
 2)

In [1]:
query = """
SELECT A.TRANS_ID,
        A.TRANS_DATE,
        A.TRANS_TIME,
        B.CATEGORY_NAME,
        A.AMOUNTS,
        A.PLACE
FROM
        LEDGER.TRANSACTION AS A
        JOIN
            LEDGER.CATEGORY AS B
            ON A.CATEGORY = B.CATEGORY_ID;
"""
cursor.execute(query)
results = cursor.fetchall()
results

NameError: name 'cursor' is not defined

In [18]:
results[0]

(1, datetime.date(2023, 1, 1), datetime.time(10, 11), '식비', 4900, '네이버파이낸셜')

In [19]:
results[0][1]

datetime.date(2023, 1, 1)

In [8]:
today = dt.date.today()
month_first = dt.date(today.year, today.month, 1).strftime('%Y-%m-%d')
query = f"""SELECT TYPE, SUM(AMOUNTS) AS AMOUNT_SUM FROM LEDGER.TRANSACTION WHERE TRANS_DATE > '{month_first}' GROUP BY TYPE;"""
cursor.execute(query)
test = cursor.fetchall()

print(test)

[(2, 652025)]


### draw a graph

In [10]:
db = psycopg2.connect(host = 'localhost', dbname = 'ledger', user = 'mato', port = 5432)
cursor = db.cursor()

In [21]:
query = """SELECT TO_CHAR(TRANS_DATE, 'YYYY-MM-DD'), SUM(AMOUNTS) AS AMOUNTS_SUM FROM LEDGER.TRANSACTION 
        WHERE EXTRACT(MONTH FROM TRANS_DATE) = EXTRACT(MONTH FROM CURRENT_DATE) 
        AND TYPE = 2
        GROUP BY TRANS_DATE;"""
cursor.execute(query)
results = cursor.fetchall()

In [25]:
pd.DataFrame(results, columns = ['DATE', 'AMOUNTS'])

,DATE,AMOUNTS
0,2023-02-01,19400
1,2023-02-02,12700
2,2023-02-03,86950
3,2023-02-04,139600
4,2023-02-05,77600
5,2023-02-06,16100
6,2023-02-07,17600
7,2023-02-08,11900
8,2023-02-09,12700
9,2023-02-10,6700


In [43]:
query = """
SELECT TO_CHAR(TRANS_DATE, 'YYYY-MM-DD'), SUM(AMOUNTS) AS AMOUNTS_SUM FROM LEDGER.TRANSACTION
WHERE TRANS_DATE >= DATE_TRUNC('MONTH', CURRENT_DATE - INTERVAL '1' MONTH)
    AND TRANS_DATE < DATE_TRUNC('MONTH', CURRENT_DATE)
    AND TYPE = 2
    GROUP BY TRANS_DATE
    ORDER BY TRANS_DATE ASC;
"""
cursor.execute(query)
results = cursor.fetchall()

In [70]:
payments_per_daily =  pd.DataFrame(results, columns = ['date', 'amounts'])

In [71]:
payments_per_daily

,date,amounts
0,2023-02-01,19400
1,2023-02-02,32100
2,2023-02-03,119050
3,2023-02-04,258650
4,2023-02-05,336250
5,2023-02-06,352350
6,2023-02-07,369950
7,2023-02-08,381850
8,2023-02-09,394550
9,2023-02-10,401250


In [62]:
query = """
SELECT TO_CHAR(TRANS_DATE, 'YYYY-MM-DD'), SUM(AMOUNTS) AS AMOUNTS_SUM FROM LEDGER.TRANSACTION
WHERE TRANS_DATE >= DATE_TRUNC('MONTH', CURRENT_DATE)
    AND TYPE = 2
    GROUP BY TRANS_DATE
    ORDER BY TRANS_DATE ASC;
"""
cursor.execute(query)
results = cursor.fetchall()

In [74]:
query = """
SELECT A.DATE, SUM(A.AMOUNTS_SUM) OVER(ORDER BY A.DATE) AS OVER_SUM
FROM (
    SELECT TO_CHAR(TRANS_DATE, 'YYYY-MM-DD') AS DATE, SUM(AMOUNTS) AS AMOUNTS_SUM 
    FROM LEDGER.TRANSACTION
    WHERE TRANS_DATE >= DATE_TRUNC('MONTH', CURRENT_DATE - INTERVAL '1' MONTH)
    AND TRANS_DATE < DATE_TRUNC('MONTH', CURRENT_DATE)
    AND TYPE = 2
    GROUP BY TRANS_DATE
    ORDER BY TRANS_DATE ASC
) AS A
;
"""
cursor.execute(query)
results = cursor.fetchall()

In [75]:
payments_per_daily =  pd.DataFrame(results, columns = ['date', 'amounts'])
payments_per_daily

,date,amounts
0,2023-01-01,8100
1,2023-01-02,104600
2,2023-01-03,118600
3,2023-01-04,201050
4,2023-01-05,225200
5,2023-01-06,242400
6,2023-01-07,296700
7,2023-01-08,353550
8,2023-01-09,487750
9,2023-01-10,565350


In [76]:
query = """
SELECT A.DATE, SUM(A.AMOUNTS_SUM) OVER(ORDER BY A.DATE) AS OVER_SUM 
FROM (SELECT TO_CHAR(TRANS_DATE, 'YYYY-MM-DD') AS DATE, SUM(AMOUNTS) AS AMOUNTS_SUM
    FROM LEDGER.TRANSACTION
    WHERE TRANS_DATE >= DATE_TRUNC('MONTH', CURRENT_DATE)
    AND TYPE = 2
    GROUP BY TRANS_DATE
    ORDER BY TRANS_DATE ASC
) AS A
;
"""
cursor.execute(query)
results = cursor.fetchall()


In [77]:
payments_per_daily = pd.merge(payments_per_daily, pd.DataFrame(results, columns = ['date2', 'amounts2']), left_index=True, right_index = True, how = 'left')
payments_per_daily

,date,amounts,date2,amounts2
0,2023-01-01,8100,2023-02-01,19400
1,2023-01-02,104600,2023-02-02,32100
2,2023-01-03,118600,2023-02-03,119050
3,2023-01-04,201050,2023-02-04,258650
4,2023-01-05,225200,2023-02-05,336250
5,2023-01-06,242400,2023-02-06,352350
6,2023-01-07,296700,2023-02-07,369950
7,2023-01-08,353550,2023-02-08,381850
8,2023-01-09,487750,2023-02-09,394550
9,2023-01-10,565350,2023-02-10,401250


In [83]:
fig = go.Figure()
fig.add_trace(go.Scatter(y = payments_per_daily['amounts'], mode = 'lines', name = 'previous'))
fig.add_trace(go.Scatter(y = payments_per_daily['amounts2'], mode = 'lines', name = 'current'))
fig.update_layout(title = 'Compare Consumption b/w previous and current months')
fig.show()

## plotly.go를 이용해 도넛차트 만들기

In [15]:
# 데이터 가져오기
query = """SELECT
            B.CATEGORY_NAME,
            SUM(A.AMOUNTS)      AS  AMOUNTS
        FROM
            LEDGER.TRANSACTION AS A
            JOIN
                LEDGER.CATEGORY AS B
                ON A.CATEGORY = B.CATEGORY_ID
        WHERE 
            A.TRANS_DATE >= DATE_TRUNC('MONTH', CURRENT_DATE - INTERVAL '1' MONTH)
        GROUP BY B.CATEGORY_NAME
        ORDER BY AMOUNTS DESC;
        """
cursor.execute(query)
results = cursor.fetchall()
print(results)

[('식비', 378200), ('주거•통신', 283430), ('취미•여가', 261128), ('경조사비', 159875), ('카페•간식', 105600), ('교통•자동차', 103850), ('편의점•마트', 96600), ('술•유흥', 80000), ('생활', 44330), ('미용', 33000), ('보험•세금•기타금융', 15000), ('쇼핑', 11100)]


In [16]:
dt = pd.DataFrame(results, columns=['category', 'value'])

In [17]:
dt

,category,value
0,식비,378200
1,주거•통신,283430
2,취미•여가,261128
3,경조사비,159875
4,카페•간식,105600
5,교통•자동차,103850
6,편의점•마트,96600
7,술•유흥,80000
8,생활,44330
9,미용,33000


In [38]:
label = list(dt['category'])
values = list(dt['value'])

fig = go.Figure(data = go.Pie(labels = label, values = values, hole = .3, pull = [0.1]))
fig.update_traces(hoverinfo = 'label+percent', textinfo = 'label+value', textfont_size = 10, marker= dict(line = dict(color = '#eeeeee', width = 2)))
fig.show()

In [39]:
def gauge_chart(achievement_rate):
    fig = go.Figure(go.Indicator(
        mode = "gauge+number",
        value = achievement_rate,
        gauge = {
            'axis': {'range': [None, 100]},
            'bar': {'color': "darkblue"},
            'steps' : [
                {'range': [0, 50], 'color': "red"},
                {'range': [50, 100], 'color': "green"}],
            'threshold': {
                'line': {'color': "red", 'width': 4},
                'thickness': 0.75,
                'value': 50}}))
    fig.show()

# Example usage:
gauge_chart(75) # displays a gauge chart with 75% achievement rate

In [40]:
import plotly.graph_objects as go

def progress_bar(achievement_rate):
    fig = go.Figure(go.Indicator(
        mode = "gauge+number",
        value = achievement_rate,
        gauge = {
            'axis': {'range': [None, 100]},
            'bar': {'color': "blue"},
            'borderwidth': 2,
            'bordercolor': "gray",
            'steps' : [
                {'range': [0, 50], 'color': "red"},
                {'range': [50, 100], 'color': "green"}],
            'threshold': {
                'line': {'color': "red", 'width': 4},
                'thickness': 0.75,
                'value': 50}}))
    fig.show()

# Example usage:
progress_bar(75) # displays a progress bar with 75% achievement rate


In [51]:
fig = go.Figure(go.Indicator(
    mode = "number+gauge+delta",
    gauge = {
        'shape': "bullet",
        'axis' : {'range': [0, 300]},
        'threshold' : {
            'line': {'color': "red", 'width': 5},
            'thickness': 0.75,
            'value': 200
        },
        'steps': [
            {'range': [0, 150], 'color': "lightgray"},
            {'range': [150, 250], 'color': "gray"}
        ],
        'bar': {'color': 'green'},
        },
    value = 220,
    align = 'center',
    delta = {'reference': 300, 'position': "left"},
    domain = {'x': [0, 1], 'y': [0, 1]},
    title = {'text': "<b>Consume</b><br><span style='color':gray; font-size:0.5em'>Kor. ₩</span>", 'font': {"size": 14}}
    
))
fig.update_layout(height = 250)
fig.show()

In [38]:
db = psycopg2.connect(host = 'localhost', dbname = 'ledger', user = 'mato', port = 5432)
cursor = db.cursor()

query = """SELECT
            SUM(AMOUNTS)      AS  AMOUNTS
        FROM
            LEDGER.TRANSACTION
        WHERE 
            TRANS_DATE >= DATE_TRUNC('MONTH', CURRENT_DATE) AND
            TYPE = 2
        ;
        """
cursor.execute(query)
results = cursor.fetchall()

In [40]:
results[0][0]

1265388

In [26]:
import plotly.graph_objects as go
import numpy as np

# Generate some random data for the sparkline
data = np.random.normal(0, 1, 50)

# Create the sparkline
fig = go.Figure(go.Scatter(
    x=np.arange(len(data)),
    y=data,
    mode="lines",
    line={"color": "blue", "width": 2},
))

fig.update_layout(
    xaxis={"visible": False},
    yaxis={"visible": False},
    margin={"l": 0, "r": 0, "t": 0, "b": 0},
)

fig.show()


In [1]:
import plotly.figure_factory as ff


In [3]:
# import plotly.plotly as py
import plotly.figure_factory as ff

import pandas as pd

data = pd.read_json('https://cdn.rawgit.com/plotly/datasets/master/BulletData.json')

fig = ff.create_bullet(
    data, markers='markers', measures='measures',
    ranges='ranges', subtitles='subtitle', titles='title',
)
# py.iplot(fig, filename='bullet chart from a dataframe')

In [4]:
fig.show()

In [5]:
data

,title,subtitle,ranges,measures,markers
0,Revenue,"US$, in thousands","[150, 225, 300]","[220, 270]",[250]
1,Profit,%,"[20, 25, 30]","[21, 23]",[26]
2,Order Size,"US$, average","[350, 500, 600]","[100, 320]",[550]
3,New Customers,count,"[1400, 2000, 2500]","[1000, 1650]",[2100]
4,Satisfaction,out of 5,"[3.5, 4.25, 5]","[3.2, 4.7]",[4.4]


In [19]:
data = (
  {"label": "Revenue", "sublabel": "US$, in thousands",
   "range": [150, 225, 300], "performance": [220,270], "point": [250]},
  {"label": "Profit", "sublabel": "%", "range": [20, 25, 30],
   "performance": [21, 23], "point": [26]},
  {"label": "Order Size", "sublabel":"US$, average","range": [350, 500, 600],
   "performance": [100,320],"point": [550]},
  {"label": "New Customers", "sublabel": "count", "range": [1400, 2000, 2500],
   "performance": [1000, 1650],"point": [2100]},
  {"label": "Satisfaction", "sublabel": "out of 5","range": [3.5, 4.25, 5],
   "performance": [3.2, 4.7], "point": [4.4]}
)
pd.DataFrame(data)

,label,sublabel,range,performance,point
0,Revenue,"US$, in thousands","[150, 225, 300]","[220, 270]",[250]
1,Profit,%,"[20, 25, 30]","[21, 23]",[26]
2,Order Size,"US$, average","[350, 500, 600]","[100, 320]",[550]
3,New Customers,count,"[1400, 2000, 2500]","[1000, 1650]",[2100]
4,Satisfaction,out of 5,"[3.5, 4.25, 5]","[3.2, 4.7]",[4.4]


In [33]:
data = (
  {"label": "Revenue", "sublabel": "US$, in thousands",
   "range": [150, 225, 300], "performance": [220,270], "point": [250]},
  {"label": "Profit", "sublabel": "%", "range": [20, 25, 30],
   "performance": [21, 23], "point": [26]},
  {"label": "Order Size", "sublabel":"US$, average","range": [350, 500, 600],
   "performance": [100,320],"point": [550]},
  {"label": "New Customers", "sublabel": "count", "range": [1400, 2000, 2500],
   "performance": [1000, 1650],"point": [2100]},
  {"label": "Satisfaction", "sublabel": "out of 5","range": [3.5, 4.25, 5],
   "performance": [3.2, 4.7], "point": [4.4]}
)

fig = ff.create_bullet(
    data, titles='label', subtitles='sublabel', markers='point',
    measures='performance', ranges='range', orientation='v',
)
fig.show()

In [51]:
import plotly.figure_factory as ff
import pandas as pd

# Create sample data
df = pd.DataFrame({
    'id': ['Metric 1', 'Metric 2', 'Metric 3'],
    'range': [ [10, 100], [10, 100], [10, 100]],
    'current': [70, 80, 90],
    'target': [80, 85, 95]
})

# Create bullet chart using Plotly Figure Factory
fig = ff.create_bullet(
    df,
    title='id',
    # marker_colors=['rgba(84,153,199,1)', 'rgba(232,90,79,1)'],
    measures='current',
    ranges = 'range',
    markers = 'target',
    orientation= 'v'
    # width=600,
    # height=400
)

# Show the bullet chart
fig.show()


TypeError: object of type 'int' has no len()

In [34]:
df

,id,current,target
0,Metric 1,70,80
1,Metric 2,80,85
2,Metric 3,90,95


In [36]:
pd.DataFrame(data)

,label,sublabel,range,performance,point
0,Revenue,"US$, in thousands","[150, 225, 300]","[220, 270]",[250]
1,Profit,%,"[20, 25, 30]","[21, 23]",[26]
2,Order Size,"US$, average","[350, 500, 600]","[100, 320]",[550]
3,New Customers,count,"[1400, 2000, 2500]","[1000, 1650]",[2100]
4,Satisfaction,out of 5,"[3.5, 4.25, 5]","[3.2, 4.7]",[4.4]


In [52]:
import collections
import math
from plotly import exceptions, optional_imports
import plotly.colors as clrs
from plotly.figure_factory import utils

import plotly
import plotly.graph_objs as go

In [54]:
data = pd.DataFrame(data)

In [57]:
data.iloc[0]["range"]

[150, 225, 300]

In [96]:
fig = go.Figure()
fig['layout'].update(
    barmode = "stack", # stack으로 해야 하나로 할 수 있음
    height = 600,
    width = 300,
    showlegend = False
    # margin = dict(l = 80)
    )
range_colors = ["rgb(200, 200, 200)", "rgb(245, 245, 245)"]
measure_colors = ["rgb(31, 119, 180)", "rgb(176, 196, 221)"]

for idx in range(len(data.iloc[0]["range"])):
    # range bars: 배경이 되는 range bars
    inter_colors = clrs.n_colors(
        range_colors[0], range_colors[1], len(data.iloc[0]["range"]), "rgb"
    )

    x = [0]
    y = [sorted(data.iloc[0]["range"])[-1 -idx]] # 거꾸로 가는 거
    bar = go.Bar(
        x = x,
        y = y,
        marker = dict(color = inter_colors[-1 - idx]),
        orientation= "v",
        base = 0,
        width = 2,
        # xaxis = "x1"
    )
    fig.add_trace(bar)

# measure bar: current value
for idx in range(len(data.iloc[0]["performance"])):
    inter_colors = clrs.n_colors(
        measure_colors[0], measure_colors[1], len(data.iloc[0]["performance"]),
        "rgb",
    )

    x = [0.5],
    y = [sorted(data.iloc[0]['performance'])[-1 - idx]]
    bar = go.Bar(
        x = x,
        y = y,
        marker = dict(color = inter_colors[-1 -idx]),
        width = 0.4,
        base = 0

    )
    fig.add_trace(bar)

fig.show()

([0.5],) [270]
([0.5],) [220]


In [ ]:
fig = go.Bar(
                x=150,
                y=0.5,
                marker=dict(color=inter_colors[-1 - idx]),
                name="measures",
                hoverinfo="x" if orientation == "h" else "y",
                orientation=orientation,
                width=0.4,
                base=0,
                xaxis="x{}".format(row + 1),
                yaxis="y{}".format(row + 1),
            )
fig.add_trace(bar)